In [2]:
!pip install tqdm

In [ ]:
import os
import numpy as np
import ast
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

path = '/kaggle/input/dataset-dikit/Dataset Dikit'

height, width, channels = 80, 112, 3

def process_single_vid(args):
    root, file, name_dir = args
    try:
        with open(os.path.join(root, file), 'r') as f:
            data_str = f.read()
        data_list = ast.literal_eval(data_str)
        data_array = np.array(data_list)
        num_frames = len(data_list)
        frames = data_array.reshape((num_frames, height, width, channels))
        return frames, name_dir
    except Exception as e:
        return None, None

file_list = []
for name_dir in os.listdir(path):
    name_path = os.path.join(path, name_dir)
    if not os.path.isdir(name_path):
        continue
    for root, dirs, files in os.walk(name_path):
        for file in files:
            if file == "data.txt":
                file_list.append((root, file, name_dir))

num_workers = min(cpu_count()-1, 8)
videos = []
labels = []

with Pool(num_workers) as pool:
    results = list(tqdm(
        pool.imap(process_single_vid, file_list),
        total = len(file_list),
        desc = "Processing",
        unit = "video"
    ))

for frames, label in results:
    if frames is not None:
        videos.append(frames)
        labels.append(label)

videos = np.array(videos)
np.save("videosCorrectDikit.npy", videos)
labels = np.array(labels)
np.save("labelsCorrectDikit.npy", labels)

Processing:  87%|████████▋ | 1087/1249 [46:58<07:12,  2.67s/video] 